In [539]:
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession

import ndjson

In [540]:
channel_graph = ChannelGraph("../channels.sample.json")
uncertainty_network = UncertaintyNetwork(channel_graph)
oracle_lightning_network = OracleLightningNetwork(channel_graph)
oracle_lightning_network_prior = OracleLightningNetwork(channel_graph)

In [541]:
source_pre = {}
for channel in oracle_lightning_network.network.edges:
    if channel[0] not in source_pre.keys():
        source_pre[channel[0]] = oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
    else:
        source_pre[channel[0]] += oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
print(source_pre)

{'A': 207500.0, 'B': 257500.0, 'C': 155000.0, 'D': 205000.0}


In [542]:
payment_set = ndjson.load(open("./same_payments_small_graph.ndjson", "r"))

In [543]:
# init simulation
c = 0
successful_payments = 0
failed_payments = 0
payment_simulation = []

# create new payment session - will later be moved before payment loop to simulate sequence of payments
sim_session = SyncSimulatedPaymentSession(oracle_lightning_network, uncertainty_network, prune_network=False)

In [544]:
for payment in payment_set:
    c += 1
    # create new payment session
    # sim_session = SyncSimulatedPaymentSession(oracle_lightning_network, uncertainty_network, prune_network=False)
    # we need to make sure we forget all learnt information on the Uncertainty Network
    # sim_session.forget_information()
    ret = sim_session.pickhardt_pay(payment["sender"], payment["receiver"], payment["amount"],
                                    mu=0, base=0, loglevel="debug")
    if ret > 0:
        successful_payments += 1
    if ret < 0:
        failed_payments += 1
    payment['success'] = ret
    payment_simulation.append(payment)

01:59:41.891 | INFO | *** new pickhardt payment ***
01:59:41.893 | INFO | Executing Payment...
01:59:41.894 | INFO | SUMMARY:
01:59:41.895 | INFO | ========
01:59:41.895 | INFO | Rounds of mcf-computations:	  1
01:59:41.896 | INFO | Number of attempts made:		  1
01:59:41.896 | INFO | Number of failed attempts:	  0
01:59:41.896 | INFO | Failure rate: 0.00% 
01:59:41.897 | INFO | total Payment lifetime (including inefficient memory management): 0.003 sec
01:59:41.897 | INFO | Learnt entropy:  0.52 bits
01:59:41.897 | INFO | fee for settlement of delivery:    0.000 sat --> 0 ppm
01:59:41.898 | INFO | used mu: 0
01:59:41.898 | INFO | Payment was successful: True
01:59:41.899 | INFO | *** new pickhardt payment ***
01:59:41.901 | INFO | Executing Payment...
01:59:41.902 | INFO | SUMMARY:
01:59:41.902 | INFO | ========
01:59:41.902 | INFO | Rounds of mcf-computations:	  1
01:59:41.903 | INFO | Number of attempts made:		  1
01:59:41.903 | INFO | Number of failed attempts:	  0
01:59:41.903 | IN

In [545]:
print(f"\n{c} payments. {successful_payments} successful, {failed_payments} failed.")


3 payments. 3 successful, 0 failed.


In [546]:
source_post = {}
for channel in oracle_lightning_network.network.edges:
    if channel[0] not in source_post.keys():
        source_post[channel[0]] = oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
    else:
        source_post[channel[0]] += oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
print(source_post)

{'A': 207500.0, 'B': 407500.0, 'C': 155000.0, 'D': 55000.0}


## Display change in outbound liquidity

In [547]:
for key in source_pre:
    print(key, source_post[key] - source_pre[key])

A 0.0
B 150000.0
C 0.0
D -150000.0


## Display channel capacities and their change from before and after as well
## as intervals in Uncertainty Network

In [548]:
for edge in oracle_lightning_network.network.edges:
    channel_init = oracle_lightning_network_prior.get_channel(edge[0], edge[1], edge[2])
    channel_now = oracle_lightning_network.get_channel(edge[0], edge[1], edge[2])
    print(f"{edge[0]}-{edge[1]} (capacity {channel_init.capacity}): {channel_init.actual_liquidity:>8.0f} to {channel_now.actual_liquidity:>8.0f}. "
          f"Intervall [{uncertainty_network.get_channel(edge[0], edge[1], edge[2]).min_liquidity:>8}; "
          f"{uncertainty_network.get_channel(edge[0], edge[1], edge[2]).max_liquidity:>8}]")

A-B (capacity 105000):    52500 to    52500. Intervall [       0;   105000]
A-C (capacity 205000):   102500 to   102500. Intervall [       0;   205000]
A-D (capacity 105000):    52500 to    52500. Intervall [       0;   105000]
B-A (capacity 105000):    52500 to    52500. Intervall [       0;   105000]
B-C (capacity 105000):    52500 to    52500. Intervall [       0;   105000]
B-D (capacity 305000):   152500 to   302500. Intervall [       0;   255000]
C-B (capacity 105000):    52500 to    52500. Intervall [       0;   105000]
C-A (capacity 205000):   102500 to   102500. Intervall [       0;   205000]
D-A (capacity 105000):    52500 to    52500. Intervall [       0;   105000]
D-B (capacity 305000):   152500 to     2500. Intervall [   50000;   305000]


## is final liquidity of channel within range of uncertainty network estimate?

In [549]:
for edge in oracle_lightning_network.network.edges:
    channel_init = oracle_lightning_network_prior.get_channel(edge[0], edge[1], edge[2])
    channel_now = oracle_lightning_network.get_channel(edge[0], edge[1], edge[2])
    uncertainty_channel_now = uncertainty_network.get_channel(edge[0], edge[1], edge[2])
    if channel_now.actual_liquidity not in range(uncertainty_channel_now.min_liquidity, uncertainty_channel_now.max_liquidity):
        print(f"PROBLEM in channel {edge[0]}-{edge[1]}. liq: {channel_now.actual_liquidity}, min: {uncertainty_channel_now.min_liquidity}, max: {uncertainty_channel_now.max_liquidity}")
    else:
        print(f"channel {edge[0]}-{edge[1]} is ok")

channel A-B is ok
channel A-C is ok
channel A-D is ok
channel B-A is ok
channel B-C is ok
PROBLEM in channel B-D. liq: 302500.0, min: 0, max: 255000
channel C-B is ok
channel C-A is ok
channel D-A is ok
PROBLEM in channel D-B. liq: 2500.0, min: 50000, max: 305000
